In [17]:
import utils as u
import preprocessing as pp
import train as tt
import visualization as viz

import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np

from keras.utils import np_utils

import spectral
from spectral import spy_colors

#tests
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [72]:
numPCAcomponents = 50
PATCH_SIZE = 5
C1 = 3*numPCAcomponents

batch_size = 128
epochs = 10

classes_authorized = [2,3,5,6,10,0,11,12,14,15]

target_names = ['Maïs- Pas de Technique de Conservation des sols',
                    'Maïs- Minimum Tillage',
                    'Herbe-Pâturage',
                    'Herbe-Arbre',
                    'Soja-Pas de Technique de Conservation des sols', 'Soja-Minimum Tillage','Soja', 'Bois',
                    'Bâtiment-Herbe-Arbre-drives',]

label_dictionary = {
    0: 'Rien',
    1: 'Maïs- Pas de Technique de Conservation des sols', 
    2: 'Maïs- Minimum Tillage',
    3: 'Herbe-Pâturage', 
    4: 'Herbe-Arbre', 
    5: 'Soja-Pas de Technique de Conservation des sols', 
    6: 'Soja-Minimum Tillage',
    7: 'Soja', 
    8: 'Bois', 
    9: 'Bâtiment-Herbe-Arbre-drives',
}

In [73]:
X, train_data, test_data = pp.loadData()

In [74]:
train_data = pp.deleteUselessClasses(train_data, classes_authorized)
test_data = pp.deleteUselessClasses(test_data, classes_authorized)

def pretty_print_count(train_data, test_data):
    unique_train = np.unique(train_data, return_counts=True)
    unique_test = np.unique(test_data, return_counts=True)
    for i in range(0, unique_train[0].shape[0]):
        print(unique_train[0][i], "=>", unique_train[1][i], "/", unique_test[1][i])
    print()
        
print("Before Shuffle: ")
pretty_print_count(train_data, test_data)
train_data, test_data = pp.shuffleTrainTest(train_data, test_data)
print("After Shuffle: ")
pretty_print_count(train_data, test_data)

Before Shuffle: 
0 => 15155 / 17753
1 => 648 / 780
2 => 751 / 79
3 => 465 / 18
4 => 492 / 238
5 => 231 / 741
6 => 1646 / 809
7 => 403 / 190
8 => 1145 / 120
9 => 89 / 297

After Shuffle: 
0 => 16441 / 16467
1 => 700 / 728
2 => 435 / 395
3 => 249 / 234
4 => 382 / 348
5 => 481 / 491
6 => 1205 / 1250
7 => 317 / 276
8 => 635 / 630
9 => 180 / 206



In [75]:
X, pca = pp.dimensionalityReduction(X, numComponents=numPCAcomponents, standardize=False)

In [276]:
def patch_1dim_split(X, train_data, test_data, PATCH_SIZE):
    padding = int((PATCH_SIZE - 1) / 2) #Patch de 3*3 = padding de 1 (centre + 1 de chaque coté)
    #X_padding = np.zeros(X)
    X_padding = np.pad(X, [(padding, padding), (padding, padding), (0, 0)], mode='constant')
    
    X_patch = np.zeros((X.shape[0] * X.shape[1], PATCH_SIZE, PATCH_SIZE, X.shape[2]))
    y_train_patch = np.zeros((train_data.shape[0] * train_data.shape[1]))
    y_test_patch = np.zeros((test_data.shape[0] * test_data.shape[1]))
    
    index = 0
    for i in range(0, X_padding.shape[0] - 2 * padding):
        for j in range(0, X_padding.shape[1] - 2 * padding):
            # This condition is for less frequent updates. 
            if i % 8 == 0 or index == (X_padding.shape[0] - 2 * padding) * (X_padding.shape[1] - 2 * padding) - 1:
                u.printProgressBar(index + 1, (X_padding.shape[0] - 2 * padding) * (X_padding.shape[1] - 2 * padding))
            patch = X_padding[i:i + 2 * padding + 1, j:j + 2 * padding + 1]
            X_patch[index, :, :, :] = patch
            y_train_patch[index] = train_data[i, j]
            y_test_patch[index] = test_data[i, j]
            index += 1
    
    print("\nCreating train/test arrays and removing zero labels...")
    u.printProgressBar(1, 7)
    X_train_patch = np.copy(X_patch)
    u.printProgressBar(2, 7)
    X_test_patch = np.copy(X_patch)
    
    u.printProgressBar(3, 7)
    X_train_patch = X_train_patch[y_train_patch > 0,:,:,:]
    u.printProgressBar(4, 7)
    X_test_patch = X_test_patch[y_test_patch > 0,:,:,:]
    u.printProgressBar(5, 7)
    y_train_patch = y_train_patch[y_train_patch > 0] - 1
    u.printProgressBar(6, 70)
    y_test_patch = y_test_patch[y_test_patch > 0] - 1
    u.printProgressBar(7, 7)
    print("Done.")
    
    return X_train_patch, X_test_patch, y_train_patch, y_test_patch

def split(X, train_data, test_data):
    X_temp = np.reshape(X, (X.shape[0] * X.shape[1], X.shape[2]))
    y_train_temp = np.reshape(train_data, (train_data.shape[0]* train_data.shape[1]));
    y_test_temp = np.reshape(test_data, (test_data.shape[0] * test_data.shape[1]));
    
    X_train = X_temp[y_train_temp > 0,:];
    y_train_temp = y_train_temp[y_train_temp > 0];
    X_test = X_temp[y_test_temp > 0];
    y_test_temp = y_test_temp[y_test_temp > 0];
    
    return X_train, X_test, y_train_temp, y_test_temp

In [277]:
X_train, X_test, y_train, y_test = split(X, train_data, test_data)


In [278]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [279]:
y_pred = neigh.predict(X_test)

In [280]:
print(classification_report(y_test, y_pred, target_names=target_names))

                                                 precision    recall  f1-score   support

Maïs- Pas de Technique de Conservation des sols       0.56      0.60      0.58       728
                          Maïs- Minimum Tillage       0.55      0.47      0.51       395
                                 Herbe-Pâturage       0.86      0.74      0.79       234
                                    Herbe-Arbre       0.80      0.96      0.87       348
 Soja-Pas de Technique de Conservation des sols       0.55      0.55      0.55       491
                           Soja-Minimum Tillage       0.65      0.74      0.69      1250
                                           Soja       0.64      0.32      0.43       276
                                           Bois       0.89      0.93      0.91       630
                    Bâtiment-Herbe-Arbre-drives       0.77      0.34      0.48       206

                                    avg / total       0.68      0.67      0.67      4558



In [281]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [282]:
X_train2 = (X_train - np.min(X_train))
X_test2 = (X_test - np.min(X_test))
multi = MultinomialNB(alpha=0.001)
multi.fit(X_train2, y_train)

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [283]:
y_pred = multi.predict(X_test2)

In [284]:
set(y_test) - set(y_pred)
print(classification_report(y_test, y_pred, target_names=target_names))

                                                 precision    recall  f1-score   support

Maïs- Pas de Technique de Conservation des sols       0.64      0.17      0.27       728
                          Maïs- Minimum Tillage       0.00      0.00      0.00       395
                                 Herbe-Pâturage       0.00      0.00      0.00       234
                                    Herbe-Arbre       1.00      0.01      0.02       348
 Soja-Pas de Technique de Conservation des sols       0.00      0.00      0.00       491
                           Soja-Minimum Tillage       0.30      0.99      0.46      1250
                                           Soja       0.00      0.00      0.00       276
                                           Bois       0.91      0.29      0.44       630
                    Bâtiment-Herbe-Arbre-drives       0.00      0.00      0.00       206

                                    avg / total       0.39      0.34      0.23      4558



/home/meryll/miniconda3/envs/eca/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [285]:
from sklearn.tree import DecisionTreeClassifier

In [286]:
dtree = DecisionTreeClassifier(max_depth=8, criterion="entropy", max_leaf_nodes=50)

In [287]:
dtree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=8,
            max_features=None, max_leaf_nodes=50,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [288]:
y_pred = dtree.predict(X_test)

In [289]:
print(classification_report(y_test, y_pred, target_names=target_names))

                                                 precision    recall  f1-score   support

Maïs- Pas de Technique de Conservation des sols       0.61      0.54      0.57       728
                          Maïs- Minimum Tillage       0.54      0.47      0.50       395
                                 Herbe-Pâturage       0.75      0.77      0.76       234
                                    Herbe-Arbre       0.85      0.90      0.88       348
 Soja-Pas de Technique de Conservation des sols       0.58      0.62      0.60       491
                           Soja-Minimum Tillage       0.64      0.75      0.69      1250
                                           Soja       0.68      0.34      0.45       276
                                           Bois       0.89      0.91      0.90       630
                    Bâtiment-Herbe-Arbre-drives       0.54      0.47      0.50       206

                                    avg / total       0.67      0.68      0.67      4558



In [290]:
svc = SVC(C=9, decision_function_shape="ovo")

In [291]:
svc.fit(X_train, y_train)

SVC(C=9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [292]:
y_pred = svc.predict(X_test)

In [293]:
print(classification_report(y_test, y_pred, target_names=target_names))

                                                 precision    recall  f1-score   support

Maïs- Pas de Technique de Conservation des sols       0.77      0.77      0.77       728
                          Maïs- Minimum Tillage       0.80      0.66      0.72       395
                                 Herbe-Pâturage       0.92      0.88      0.90       234
                                    Herbe-Arbre       0.93      0.97      0.95       348
 Soja-Pas de Technique de Conservation des sols       0.80      0.77      0.78       491
                           Soja-Minimum Tillage       0.79      0.85      0.82      1250
                                           Soja       0.71      0.76      0.74       276
                                           Bois       0.93      0.97      0.95       630
                    Bâtiment-Herbe-Arbre-drives       0.81      0.62      0.70       206

                                    avg / total       0.82      0.82      0.82      4558

